# Silver - Merging Datasets

## 0. Setup

In [1]:
debugging = True

In [2]:
import findspark

findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import types as t

from utils import setup_spark

spark: SparkSession = setup_spark(title="Silver Merge")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/28 16:20:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from schemas import schema_chicago_silver
from schemas import schema_nyc_silver
from shared import PATH_SILVER_CHICAGO
from shared import PATH_SILVER_NYC

silver_nyc = spark.read.schema(schema_nyc_silver).parquet(PATH_SILVER_NYC).rdd
silver_chicago = spark.read.schema(schema_chicago_silver).parquet(PATH_SILVER_CHICAGO).rdd

## 1. Prerequisites

In [5]:
from shared import City
from shared import PaymentType
from shared import PaymentTypeCHI
from shared import PaymentTypeNYC

In [6]:
def get_duration_minutes_nyc(row: t.Row) -> int:
    duration = row.tpep_dropoff_datetime - row.tpep_pickup_datetime
    return round(duration.total_seconds() // 60)


def get_duration_minutes_chi(row: t.Row) -> int:
    return round(row.trip_seconds // 60)


def get_speed_nyc(row: t.Row) -> float | None:
    seconds: int = (row.tpep_dropoff_datetime - row.tpep_pickup_datetime).total_seconds()
    if seconds == 0:
        return None
    return row.trip_distance / seconds * 3600


def get_speed_chi(row: t.Row) -> float | None:
    seconds: int = row.trip_seconds
    if seconds == 0:
        return None
    return row.trip_miles / seconds * 3600


def get_tip_pct_nyc(row: t.Row) -> float | None:
    if row.total_amount == 0:
        return None
    return row.tip_amount / row.total_amount * 100


def get_tip_pct_chi(row: t.Row) -> float | None:
    if row.trip_total == 0:
        return None
    return row.tips / row.trip_total * 100


def get_payment_type_nyc(row: t.Row) -> int:
    match PaymentTypeNYC(row.payment_type):
        case PaymentTypeNYC.CASH:
            return PaymentType.CASH.value
        case PaymentTypeNYC.CREDIT_CARD:
            return PaymentType.CREDIT_CARD.value
        case PaymentTypeNYC.NO_CHARGE:
            return PaymentType.NO_CHARGE.value
        case PaymentTypeNYC.DISPUTE:
            return PaymentType.DISPUTE.value
        case PaymentTypeNYC.UNKNOWN:
            return PaymentType.UNKNOWN.value
        case PaymentTypeNYC.VOIDED_TRIP:
            return PaymentType.VOIDED_TRIP_NYC.value
        case _:
            raise ValueError("Unexpected NYC payment type")


def get_payment_type_chi(row: t.Row) -> int:
    match PaymentTypeCHI(row.payment_type):
        case PaymentTypeCHI.cash:
            return PaymentType.CASH.value
        case PaymentTypeCHI.credit_card:
            return PaymentType.CREDIT_CARD.value
        case PaymentTypeCHI.no_charge:
            return PaymentType.NO_CHARGE.value
        case PaymentTypeCHI.unknown:
            return PaymentType.UNKNOWN.value
        case PaymentTypeCHI.dispute:
            return PaymentType.DISPUTE.value
        case PaymentTypeCHI.pcard:
            return PaymentType.PRCARD_CHI.value
        case PaymentTypeCHI.prcard:
            return PaymentType.PRCARD_CHI.value
        case _:
            raise ValueError("Unexptected chicago payment type")


In [7]:
def convert_nyc_row(row: t.Row) -> t.Row:
    return t.Row(
        # ADDED
        city=City.NYC.value,
        duration_minutes=get_duration_minutes_nyc(row),
        speed_mph=get_speed_nyc(row),
        tip_pct=get_tip_pct_nyc(row),
        # BOTH
        datetime_start=row.tpep_pickup_datetime,
        datetime_end=row.tpep_dropoff_datetime,
        distance_miles=row.trip_distance,
        payment_type=get_payment_type_nyc(row),
        fare_amount=row.fare_amount,
        extra_amount=row.extra,
        tip_amount=row.tip_amount,
        tolls_amount=row.tolls_amount,
        total_amount=row.total_amount,
        # NYC
        vendor_nyc=row.VendorID,
        passenger_count_nyc=row.passenger_count,
        rate_code_nyc=row.RateCodeID,
        mta_tax_amount_nyc=row.mta_tax,
        improvement_surcharge_amount_nyc=row.improvement_surcharge,
        # CHICAGO
        taxi_id_chi=None,
        community_area_pickup_chi=None,
        community_area_dropoff_chi=None,
        company_chi=None,
    )


def convert_chicago_row(row: t.Row) -> t.Row:
    return t.Row(
        # ADDED
        city=City.CHICAGO.value,
        duration_minutes=get_duration_minutes_chi(row),
        speed_mph=get_speed_chi(row),
        tip_pct=get_tip_pct_chi(row),
        # BOTH
        datetime_start=row.trip_start_timestamp,
        datetime_end=row.trip_end_timestamp,
        distance_miles=row.trip_miles,
        payment_type=get_payment_type_chi(row),
        fare_amount=row.fare,
        extra_amount=row.extras,
        tip_amount=row.tips,
        tolls_amount=row.tolls,
        total_amount=row.trip_total,
        # NYC
        vendor_nyc=None,
        passenger_count_nyc=None,
        rate_code_nyc=None,
        mta_tax_amount_nyc=None,
        improvement_surcharge_amount_nyc=None,
        # CHICAGO
        taxi_id_chi=row.taxi_id,
        community_area_pickup_chi=row.pickup_community_area,
        community_area_dropoff_chi=row.dropoff_community_area,
        company_chi=row.company,
    )

## 2. Convert rows

In [8]:
silver_nyc_converted = silver_nyc.map(convert_nyc_row)
silver_chicago_converted = silver_chicago.map(convert_chicago_row)

## 3. Save results

In [9]:
from schemas import schema_merged_silver
from shared import PATH_SILVER_MERGED

silver_nyc_converted.toDF(schema=schema_merged_silver).write.parquet(PATH_SILVER_MERGED, mode="overwrite")
silver_chicago_converted.toDF(schema=schema_merged_silver).write.parquet(PATH_SILVER_MERGED, mode="append")

In [10]:
spark.stop()